# A Comprehensive Study of Multiple Encoding Techniques on Low Resource Embeddings for Character-Level NLP

## Flexible Universal Character Level Encoding method for Text in NLP

    Leonardo M. Rocha
    leo <dot> m <dot> rocha <at> gmail <dot> com
    

## Abstract

Currently NLP deals with *Out of Vocabulary* (OOV) in different ways, this leads to several non-necessarilly efficient ways of pre-processing NLP datasets to be able to deal with 
In this work we present **Segment-Multihot-Encoding** (SME) a technique that deals with OOV and allows to encode all possible symbols in a computationally efficient manner to encode all (or part) of the UTF-8 domain in a fixed multi-hot encoding that can be further compressed by overfitting. This technique eliminates the need of complex and compute consuming pre-processing replacing it for a much more simple one that works for *any* dataset. This work focuses on being able to encode a symbol, as once it is encoded the network can be fine-tunned later to handle previously unseen ones.

This work presents the SME technique for UTF8 and we call it **SME-UTF8**, the source code and encoding vectors are also given with usage examples.

There is an extra advantage of this methodology is that with deterministic and defined process with small representation size can be implemented efficiently in hardware.

This is also presented directly as an ipython notebook to also be Executable in the places that is needed. We call this, the Executable Paper and the idea is to improve reproducibility.



## Introduction and Related Work

Currently for NLP tasks there is the need to first analyze the input domain and encode it to deal with *Out of Vocabulary* (OOV) words or symbols and Polysemy. 

It is important to separate (and we do in this work) the encoding part (to be able to represent the symbol) from the learning to use those symbols (the network to be able to do something useful with it) as this work focuses solely on being able to encode all feasible symbols in a defined text encoding domain. In this case the work is done for UTF-8 which is the most used text encoding in the web [94.6% according to w3techs](https://w3techs.com/technologies/details/en-utf8).



Diverse techniques deal differently with OOV, ranging from techniques that can not deal with them, like [GloVe - Pennington et al. 2014](https://nlp.stanford.edu/pubs/glove.pdf) or [Word2Vec - Milikov et al. 2013](https://arxiv.org/abs/1301.3781) to others such as [Universal Sentence Encoder -Cer et al. 2018](https://arxiv.org/abs/1803.11175) or the one for FastText can encode OOV with subword embeddings
One of the most used techniques is [Byte Pair Encoding from Neural Machine Translation of Rare Words with Subword Units Sennrich et al. 2015](https://arxiv.org/abs/1508.07909) which has the advantage of compressing the input size hence accelerate training compared to a full character level input on the current SoTA.


[ELMo - Peters et al. 2018](https://arxiv.org/abs/1802.05365)

[Transformer - ]()
[ULM-FiT - ]()
[BERT - ]()
[AlBERT - ]()
[CamemBERT]()


... TODO more papers and references here



All current methods deal with subdomains of the possible inputs available, which for most tasks is enough, nevertheless the weakness is that they can not deal with **all** posisble input symbols, which for the current study is UTF-8.

In the case of continual learning the need to add new symbols is a given, be it due to adding new domain in the same language, or new languages

The goal of this work is to try to set a more standard way to deal with all possible symbols in a defined encoding standard.

This work analyzes UTF-8 encoding and presents a technique to be able to encode part or all of the UTF-8 domain in an computationally efficient way. The same technique can be used for other text encodings without any modification, and as utf-8 is a superset of other encodings (ASCII, Unicode, ...) the same matrix encoding can be applied without any modification in those datasets.

There is also another point to say about this computational complexity, all the current SoTA methods are trained in clusters (and wiht prices) that are unavailable to most users. The current work is part or a larger work on trying to get enough permormance in commercially available (and relatively accessible) single GPUs for end users (being at the current time the NVidia RTX2080ti one of the more computationally strong cards in the market).


### Sections

This paper deals first with an analysis of the UTF-8 encoding

Then goes to deal with the construction of the multi-hot code proposed

After works on compressing the multi-hot code with Overfitting (yes, OVERFITTING)

Then goes to the evaluation of the codes and compare results with other encoding methods

Finally goes to the conclussion and future work.


## UTF-8 Analysis

### One-Hot encoding

[One-hot encoding](https://en.wikipedia.org/wiki/One-hot) is one of the most used to encode cathegorical variables, in the case of State of the Art NLP tasks is used to encode the input symbols, this is computationally expensive and the goal here is to reduce this complexity leaving memory and computational space for other more complex tasks in the network.

### Number of code-points

As this file tries to encode all the characters possible by utf-8 we have to check the feasible number so:

From [Wikipedia utf-8](https://en.wikipedia.org/wiki/UTF-8)

UTF-8 is a variable width character encoding capable of encoding all **1,112,064**.

$$ 17×2^{16} = 1114112 $$ code points minus 2,048 technically-invalid surrogate code points

This is, if encoding with one-hot we would need 1.1M parameters per neuron in the input layer, which is expensive. The goal is to reduce this complexity (which we argue is unnecessary) by orders of magnitude.

## UTF-8 structure and Encoding Details

Since the entire utf-8 univers is NOT the entire $2^{32}$ domain, but there are limitations explained in [the utf-8 description](https://en.wikipedia.org/wiki/UTF-8)

| Number of bytes | Bits for code point | First code point | Last code point | Byte 1   | Byte 2   | Byte 3   | Byte 4   |
|----------------|--------------------|-----------------|----------------|----------|----------|----------|----------|
| 1              | 7                  | U+0000          | U+007F         | 0xxxxxxx |          |          |          |
| 2              | 11                 | U+0080          | U+07FF         | 110xxxxx | 10xxxxxx |          |          |
| 3              | 16                 | U+0800          | U+FFFF         | 1110xxxx | 10xxxxxx | 10xxxxxx |          |
| 4              | 21                 | U+10000         | U+10FFFF       | 11110xxx | 10xxxxxx | 10xxxxxx | 10xxxxxx |

The UTF-8 code is formed by 4 segments, we will refer to this often during the current work.

The thing is that the number of elements in the table should be at most $2^{21}$, There is only the need to create a index that can handle the 4 cases which can be done with 4 different conversion tables.


In fact it is possible to just cut the utf-8 value in chunks and do one-hot per different parts:
- there are only 4 segment ranges, that can be coded to add redundancy in one-hot also add there either hamming or other ECC
- the largest value is for 8 bits -> 256 values
- the others contain 6 bits -> 64 values
The prefix of each can be taken away and replaced by the initial one-hot

So a complete code would be:  $ 4 + 256 + 64 + 64 + 64 = 324 $

Instead of having a vector of dimension 1,112,064 to encode any utf-8 value, one with dimension 452 would be able to encode everything in the utf-8 domain.

This embedding can stil be reduced but should be sparse enough already to make a good input, the goal here is to have sparse vector that makes each vector far enough of the others, at least by one dimension. Adding the redundancy code (the first 4 dimensions) allows to make distance even bigger for vectors that should be further appart taking into account the locality of the utf-8 encoding (each character set is close to the ones used with it, segment 3 encodes mostly CJK Chinese-Japanese-Korean).

#### Notes
It is worth noting here that the first author has also experience in communications which allowed during the curse of this research the analysis of multiple Error Correcting Codes (ECCs) and different kinds of encoding (for example encoding as a Fourier Series), the conclusion is that even if the one-hot is the best in distance, other codes can be used and a multi-hot sparse is the simplest to implement (and fastest to encode). As a note, one pending task is to analyze ECCs in an end to end manner for a neural network. Some of these analysis (without proper formating) can be found in the anex notebooks folders of this repository or at [text subfolder in the minibrain project](https://github.com/leomrocha/minibrain/tree/master/predictors/sequence/text) where most of the experimental code is located.

## Encoding details

### UTF-8 Segments
To cut even more memory consumption the table can be generated for 1-4 segments of the utf-8 code, taking into account that the 4th segment is mostly composed of:
* Supplementary Multilingual Plane (SMP) of historic scripts
* Supplementary Special-purpose Plane (SSP)
* Private Use Areas (SSU)
* Invalid Codes

We can safely ignore this 4th segment (for the purpose of this article and most usages) which adds to most of the code-points

If CJK, Indic and some Miscelaneous Symbols are not (and will not be) needed then the 3rd segment can be safely ignored too reducing even more the memory consumption of the application

So the result would be:

| Segment | # of code points | First index | Last index | Vector Size | # code exceptions | Size (MB) |Matrix Size (MB) | Sparse Size (MB) |
|---------------|-----------------------|-------------|-------------|------------|-------------------|-------------------|-----------|-----------|
| 4             | 1107904             | 61440        | 1107904     | 452 | 790656  | 11538.59   | 3820.59 | 83.59 |
| 3             | 59328               | 2112         | 61439       | 388 | 4224    | 530.71     | 175.62  | 3.59  |
| 2             | 1984                | 128          | 2111        | 324 | 128     |  14.84     | 4.90    | 0.09  |
| 1             | 128                 | 0            | 127         | 260 | 0       |  0.77      | 0.25    | 0.005  |

Where:
* Segment: number of segments used from utf-8 to generate the code
* \# of Code Points: The total encoded code points generated
* Vector Size: The embedding vector size
* First / Last Index: corresponds to the segment first and last index in the embedding matrix
* \# Code Exceptions: Number of code exceptions during encoding with Python, notice that we use the standard library for this.
* Size (MB): Size of the embedding matrix and conversion dictionaries (from-to code) once saved in disk in Dense mode
* Matrix Size (MB): Size of the embedding matrix in disk in Dense Mode
* Sparse Size (MB): Size of the embedding matrix in disk in Sparse mode


Notice that the code for 1 segment corresponds to one-hot encoding of ASCII encoding (plus the vector of size 4 that we don't modify in any case)

## Signaling the Start and End of a Sequence 

To this end we can use the codes available in the utf-8 and add the mapping to the encoding and decoding dictionaries (not the matrix), The first 0x20 codes in UTF-8 signal different communication an control codes, we can use those same for our NLP purposes or we could choose the invalid codes at 0xC0 and 0xC1 or codes larger than U+10FFFF (at the end of segment 4 as per in our vocabulary for this paper).

In order to avoid any issues and as we don't count on using the current models for communication protocols but only for NLP purposes (this encoding could also be used for allowing the network to deal directly with network communication protocols) we decide to use the control codes at the beginning of the block of the first segment

To this end there are 3 codes that are re-mapped by design and signaled by the following:
* \<start\>: 0x02 - STX Start of Text
* \<end\>: 0x03 - ETX End of Text
* \<unk\>": 0x15 - NAK  Negative Acknowledge

The \<unk\> (Unknown) element should not be used as per design there should be no unrepresented symbols in the code design, but is added for completion and in case of future use.

The mapping is created in a way to be as close as possible to the original meaning of the control symbols.


### Codes Creation

This section is dedicated to code execution and measurements to fill the table above

In [34]:
from paper_main import create_measure_tables
create_measure_tables()

number of codes =  128
number of code_exceptions =  0
number of codes =  1984
number of code_exceptions =  128
number of codes =  59328
number of code_exceptions =  4224
number of codes =  1107904
number of code_exceptions =  790656
| Segments | exec_time (sec) |  matrix_shape | Size in Disk (MB): | Matrix Size in Disk (MB):            | Sparse Matrix Size in Disk (MB): |code path
| 1 | 0.012 | (128, 260) | 0.78 | 0.25 | 0.00 | codes/utf8_codes-1seg.pkl |
| 2 | 0.065 | (1984, 324) | 14.85 | 4.90 | 0.09 | codes/utf8_codes-2seg.pkl |
| 3 | 1.952 | (59328, 388) | 530.71 | 175.62 | 3.59 | codes/utf8_codes-3seg.pkl |
| 4 | 39.106 | (1107904, 452) | 11538.59 | 3820.59 | 83.59 | codes/utf8_codes-4seg.pkl |


### Observations

### Execution of previous results

The execution of all the previous code is done in a single thread of an Intel i7 7700.

#### Embedding Sizes

Size of the embedding matrix grows with the number of code points and embedding size, observing the size of the matrices, the sparse representation of them is negligeable in comparison with current models. 

NVidia provides support for sparse operations with [cuSPARSE ](https://developer.nvidia.com/cusparse) ([documentation](https://docs.nvidia.com/cuda/cusparse/index.html)) which means we can use these matrices. 

Nevertheless many applications work in dense mode and if is the case working with less than the 4 segments would be advisable. The 3 segments embedding provides enough representation for all languages on earth and the 2 segment one support most languages already as stated in a previous section.

#### Execution Time

As seen in the code execution above, even if vectors are big to keep saved and download, the creation of the vectors is defined and can be recreated with less than a minute execution in a single thread of an off-the-shelf CPU.


### PC Configuration:

#### Hardware

    intel i7 7700
    64GB of RAM
    GPU-0 GTX1080 -> runs the GUI and other tasks, sometimes used for train and testing
    GPU-1 RTX2080ti -> only used for computation

#### Software

    Cuda V10.1
    Pytorch V1.3
    Python 3.7


## Overfitting Compression

In the literature overfitting is an evil creature, but in this case, as we know the entire domain, we are going to use it to our advantage with overfitting the sparse input (the multi-hot encoded vector) into a smaller embedding vector than the input, the goal is lossless compression here.

This is done to be able to make a more informed decision at the end of the study and show the comparative results.

Once the network is trained (basically an overfitted autoencoder), a new encoding matrix is generating making each element of the input domain pass by the autoencoder and getting the latent vector, which is used to generate a matrix that can be given as Embedding directly to the network.

As the UTF-8 coding uses a max of 4 bytes for the code representation, there is at least the need to use vectors of size 32. The best code for this would be directly using the utf-8 code as the embedding (which we should also test as input to be able to compare the results)

But then, any vector representation that handles the complete domain must be at least of dimension 32, here we'll test several dimensions for each number of codes, the representation of 1 segment coding is just done for completion, the one for 2 segments coding might be useful but having only 1984 elements a one-hot encoding does not pose a big problem with current resources, the code starts to be more interesting for 3 segment and 4 segment coding.



The training is done with the following configuration:

    Batch Size: Size of all the simbols, each batch contains once every symbol in the domain
    Network Configuration: Autoencoder
    Loss: CrossEntropy
    
And we measure:

    Output Vector Embedding Size
    Execution Time
    Matrix Size on Disk (here only the dense matrix is taken into account as there should be close to no sparsity)
    

The experiments on overfitting were run on different vector sizes from 32 to 128 for encodings using 3 and 4 segments

**TODO do the runs again (with a better and more clean script) and put here the results**

## Sparse Codes

In this section we develop a methodology and different codes with different vector sizes and distances. All the generated codes are saved and later used to test in NLP tasks.

Two different ideas are used to create the sparse codes, the first one is choose k from N, the second idea is to generate a multi-hot with the combination of different smaller codes of co-prime sizes, this leads to longer cycles on the combination of the codes

Although any code order might be enough we look for repeatable, predictable and deterministic process to reach to the same values each time we recompute the code.

## Sparse Codes, Choosing *k* of *N*

For completion, this study also deals with different sparse coding techniques, the basic idea will be choosing the 

We need to basically do the following: ${N\choose k}$ 

Where $ 32 <= N <= M$ and defining $M$ as the maximum value of the desired vector dimension 

and $ k $ should be minimized to augment the sparcity of the vector as much as possible

Also We can again add some redundancy as in the previous multi-hot code. i.e. the first 4 elements should indicate which of the UTF-8 segment are being used for the code-point.

${N\choose k} = \frac{(n)!}{k!(n-k)!} $ 

The vector sizes are explored here:

In [37]:
import numpy as np
NCODES = [128, 1984, 59328, 1107904]
def sparse_Nk_dimension_analysis():
    # find the minimum N and k for which the condition is filled for the different codes
    results = []
    for code_points in NCODES:
        for k in [2, 3, 4, 5, 6]:
            for N in range(10, 256):
                v = int(np.prod(list(range(N, N - k, -1))) / np.prod(list(range(1, k + 1))))
                if v > code_points:
                    # print("code_size={}; N={},k={}".format(v, N, k))
                    results.append((code_points, v, N, k, '{:.3f}'.format(k/N)))
                    break
    return results

In [38]:
%%time
results = sparse_Nk_dimension_analysis()
len(results)

CPU times: user 24.5 ms, sys: 166 µs, total: 24.7 ms
Wall time: 24.2 ms


18

In [39]:
# results: (code points needed, possible code points, vector size, number of  ones in code, sparsity ratio)
results

[(128, 136, 17, 2, '0.118'),
 (128, 165, 11, 3, '0.273'),
 (128, 210, 10, 4, '0.400'),
 (128, 252, 10, 5, '0.500'),
 (128, 210, 10, 6, '0.600'),
 (1984, 2016, 64, 2, '0.031'),
 (1984, 2024, 24, 3, '0.125'),
 (1984, 2380, 17, 4, '0.235'),
 (1984, 2002, 14, 5, '0.357'),
 (1984, 3003, 14, 6, '0.429'),
 (59328, 59640, 72, 3, '0.042'),
 (59328, 66045, 37, 4, '0.108'),
 (59328, 65780, 26, 5, '0.192'),
 (59328, 74613, 22, 6, '0.273'),
 (1107904, 1125180, 190, 3, '0.016'),
 (1107904, 1150626, 74, 4, '0.054'),
 (1107904, 1221759, 45, 5, '0.111'),
 (1107904, 1344904, 34, 6, '0.176')]

We can observe that the size of the vectors on these representation are much smaller than the manual one-hot code by segment created before. We can again use the same technique as in the first part of adding first 4 vector elements that represent the segment to which the symbol belongs.

There is another point too to take into account, the size of the vector is important as hardware is more adapted for certain sizes, it also has consequences on the kind of techniques can be done, for example groups convolution.

It is convenient then to have vector sizes of powers of two and also multiple of 96 (tensor operation sizes in NVidia tensor cores are of size 96)

## Sparse Codes, multiple joint co-prime codes

The idea here is to create multiple ohe-hot codes, each code of a prime size (which is the simplest way to choose co-primes), then joining the codes.

Again we look for combinations that minimize the vector size.

In [115]:
PRIMES = [2, 3, 5, 7, 11, 13, 17, 19, 23, 27, 29, 31, 37, 41, 43, 47, 49, ]
from itertools import combinations


In [139]:
np.array(list(combinations(PRIMES, 6)))


array([[ 2,  3,  5,  7, 11, 13],
       [ 2,  3,  5,  7, 11, 17],
       [ 2,  3,  5,  7, 11, 19],
       ...,
       [23, 27, 31, 37, 41, 43],
       [23, 29, 31, 37, 41, 43],
       [27, 29, 31, 37, 41, 43]])

In [243]:
def multihot_primes():
    all_codes = []
    for i in range(2, 7):
        print("Start counting for sparsity = ", i)
        arr = list(combinations(PRIMES, i))
        for a in arr:
            ll = len(a)
            ncodes = np.prod(a)
            vsizes = np.sum(a)
            all_codes.append((a, ll, vsizes, ncodes))
    all_codes = sorted(all_codes, key=lambda x: x[-2])
    return all_codes


In [244]:
%%time
C= multihot_primes()

Start counting for sparsity =  2
Start counting for sparsity =  3
Start counting for sparsity =  4
Start counting for sparsity =  5
Start counting for sparsity =  6
CPU times: user 110 ms, sys: 0 ns, total: 110 ms
Wall time: 109 ms


In [245]:
%%time
c,c1s,c2s,c3s,c4s = multihot_primes()

Start counting for sparsity =  2
Start counting for sparsity =  3
Start counting for sparsity =  4
Start counting for sparsity =  5
Start counting for sparsity =  6
CPU times: user 133 ms, sys: 3.83 ms, total: 137 ms
Wall time: 135 ms


In [254]:
all_codes = c

In [255]:
codes_1seg = [ i for i in all_codes if i[-1] > NCODES[0] and i[-1] < NCODES[3]*2]
codes_2seg = [ i for i in all_codes if i[-1] > NCODES[1] and i[-1] < NCODES[3]*2]
codes_2seg = [ i for i in all_codes if i[-1] > NCODES[2] and i[-1] < NCODES[3]*2]
codes_2seg = [ i for i in all_codes if i[-1] > NCODES[3] and i[-1] < NCODES[3]*2]

In [247]:
len(c)

9933

In [230]:
NCODES

[128, 1984, 59328, 1107904]

In [252]:
c3s

[]

In [93]:
mp

[array([[  2,   3,   5,   6],
        [  2,   5,   7,  10],
        [  3,   5,   8,  15],
        [  2,   7,   9,  14],
        [  3,   7,  10,  21],
        [  5,   7,  12,  35],
        [  2,  11,  13,  22],
        [  7,  11,  18,  77],
        [  5,  11,  16,  55],
        [  3,  11,  14,  33],
        [ 11,  13,  24, 143],
        [  7,  13,  20,  91],
        [  5,  13,  18,  65],
        [  3,  13,  16,  39],
        [  2,  13,  15,  26],
        [ 13,  17,  30, 221],
        [  5,  17,  22,  85],
        [ 11,  17,  28, 187],
        [  2,  17,  19,  34],
        [  7,  17,  24, 119],
        [  3,  17,  20,  51],
        [  3,  19,  22,  57],
        [ 17,  19,  36, 323],
        [ 13,  19,  32, 247],
        [ 11,  19,  30, 209],
        [  7,  19,  26, 133],
        [  2,  19,  21,  38],
        [  5,  19,  24,  95],
        [  5,  23,  28, 115],
        [ 11,  23,  34, 253],
        [ 17,  23,  40, 391],
        [  2,  23,  25,  46],
        [ 13,  23,  36, 299],
        [ 

## Decoding

Decoding the vectors can be done by cosine similarity (or any other method of vector similarity), in this case we use [faiss](https://github.com/facebookresearch/faiss) library from Facebook AI Research



## Method Validation

To validate this method is sufficient to show that the performance of a network does not decay with compared to one-hot in several tasks. This article deals with this in a restricted environment

There are a few key points to measure:

* Pre-processing time (dataset) for each method
* Network Performance 
* Network Size (total and trainable parameters)
* Network Memory Consumption
* Network Training Time

To this end simple enough NLP tasks will be tackled such as the training and testing time is not excesive (running on a single end user graphic GPU card, in this case an RTX2080ti).

The tasks will be evaluated on the same (except of the first embedding layer) networks with different encoding, to be able to compare networks all will be done at character level. For completeness some otehr methods also will be evaluated, mainly BPE which is currently used in most SoTA papers.

## Results and Discussion

The TODO here

## Conclussion

This work shows a different take of the current approach in how to deal with input coding for Natural Language Processing tasks on Deep Learning Networks. 

This work shows that is possible to reduce computational complexity *and* add representational capability to a deep neural network without loss of performance and making pre-processing easier .... BLBLBLABLABLABLABLABLABLABLABLA TODO here.

## Future Work:

This study is the first part of a deeper study on how to make networks train faster and be able to run on consumer grade GPUs in a competitive way (even if they are not SoTA). ..... TODO here